![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

 First, let's examine the data structure

In [11]:
import pandas as pd
import numpy as np
from datetime import datetime
# Start coding here...

In [3]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64


In [4]:
# Check initial structure
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

Clean the Client Data

In [5]:
# Create client dataframe
client = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage']].copy()

# Clean job column - replace dots with underscores
client['job'] = client['job'].str.replace('.', '_')

# Clean education column
client['education'] = client['education'].str.replace('.', '_')
client['education'] = client['education'].replace('unknown', np.nan)



In [6]:
# Handle credit_default - we need to check if 'unknown' should be treated differently
print("\nCredit default value counts before cleaning:")
print(client['credit_default'].value_counts())



Credit default value counts before cleaning:
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64


In [7]:
# Convert credit_default to boolean (1 for "yes", 0 otherwise)
client['credit_default'] = client['credit_default'].apply(lambda x: 1 if x == 'yes' else 0)

# Handle mortgage - similar to credit_default
print("\nMortgage value counts before cleaning:")
print(client['mortgage'].value_counts())




Mortgage value counts before cleaning:
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64


In [8]:
# Convert mortgage to boolean (1 for "yes", 0 otherwise)
client['mortgage'] = client['mortgage'].apply(lambda x: 1 if x == 'yes' else 0)

# Verify client data
print("\nClient data after cleaning:")
print(client.head())
print(client.dtypes)


Client data after cleaning:
   client_id  age        job  marital    education  credit_default  mortgage
0          0   56  housemaid  married     basic_4y               0         0
1          1   57   services  married  high_school               0         0
2          2   37   services  married  high_school               0         1
3          3   40     admin_  married     basic_6y               0         0
4          4   56   services  married  high_school               0         0
client_id          int64
age                int64
job               object
marital           object
education         object
credit_default     int64
mortgage           int64
dtype: object


In [19]:
# Save to CSV
client.to_csv('client.csv', index=False)
print("\nCleaned client data saved to client.csv")


Cleaned client data saved to client.csv


Key Techniques:

String Standardization: Replaced special characters in job/education fields

Missing Data Handling: Converted 'unknown' education values to NaN

Boolean Conversion: Transformed categorical yes/no to binary 1/0

Data Validation: Verified value distributions before/after transformations

Clean the Campaign Data

In [12]:
# Create campaign dataframe using the exact column names from your DataFrame
campaign = df[['client_id', 'number_contacts', 'contact_duration', 
              'previous_campaign_contacts', 'previous_outcome', 
              'campaign_outcome', 'month', 'day']].copy()



In [13]:
# 1. Convert outcome columns to boolean
print("Value counts before conversion:")
print("Previous outcome:\n", campaign['previous_outcome'].value_counts())
print("\nCampaign outcome:\n", campaign['campaign_outcome'].value_counts())

campaign['previous_outcome'] = campaign['previous_outcome'].apply(
    lambda x: 1 if x == 'success' else 0
)
campaign['campaign_outcome'] = campaign['campaign_outcome'].apply(
    lambda x: 1 if x == 'yes' else 0
)



Value counts before conversion:
Previous outcome:
 nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64

Campaign outcome:
 no     36548
yes     4640
Name: campaign_outcome, dtype: int64


In [14]:
# 2. Create last_contact_date with proper error handling
def create_date(row):
    try:
        # Convert month abbreviation to month number
        month_num = datetime.strptime(row['month'], '%b').month
        return datetime(2022, month_num, row['day'])
    except:
        return pd.NaT

campaign['last_contact_date'] = campaign.apply(create_date, axis=1)

# Check for invalid dates
invalid_dates = campaign[campaign['last_contact_date'].isna()]
if not invalid_dates.empty:
    print(f"\nWarning: {len(invalid_dates)} invalid dates found")
    # Replace invalid dates with the most common valid date
    common_date = campaign['last_contact_date'].mode()[0]
    campaign['last_contact_date'] = campaign['last_contact_date'].fillna(common_date)
    print(f"Replaced invalid dates with: {common_date}")



In [15]:
# 3. Ensure proper data types
campaign = campaign.astype({
    'client_id': 'int64',
    'number_contacts': 'int64',
    'contact_duration': 'int64',
    'previous_campaign_contacts': 'int64',
    'previous_outcome': 'int8',
    'campaign_outcome': 'int8'
})



In [16]:
# 4. Select final columns in required order
campaign_final = campaign[[
    'client_id',
    'number_contacts',
    'contact_duration',
    'previous_campaign_contacts',
    'previous_outcome',
    'campaign_outcome',
    'last_contact_date'
]]



In [17]:
# 5. Data validation
print("\nFinal data validation:")
print("Data types:\n", campaign_final.dtypes)
print("\nMissing values:\n", campaign_final.isna().sum())
print("\nValue ranges:")
print("Number contacts:", campaign_final['number_contacts'].min(), "-", campaign_final['number_contacts'].max())
print("Contact duration:", campaign_final['contact_duration'].min(), "-", campaign_final['contact_duration'].max())
print("Previous contacts:", campaign_final['previous_campaign_contacts'].min(), "-", campaign_final['previous_campaign_contacts'].max())


Final data validation:
Data types:
 client_id                              int64
number_contacts                        int64
contact_duration                       int64
previous_campaign_contacts             int64
previous_outcome                        int8
campaign_outcome                        int8
last_contact_date             datetime64[ns]
dtype: object

Missing values:
 client_id                     0
number_contacts               0
contact_duration              0
previous_campaign_contacts    0
previous_outcome              0
campaign_outcome              0
last_contact_date             0
dtype: int64

Value ranges:
Number contacts: 1 - 56
Contact duration: 0 - 4918
Previous contacts: 0 - 7


In [18]:
# 6. Save to CSV
campaign_final.to_csv('campaign.csv', index=False)
print("\nCleaned campaign data saved to campaign.csv")


Cleaned campaign data saved to campaign.csv


Key Techniques:

Date Parsing: Robust handling of month abbreviations and invalid dates

Memory Optimization: Used int8 for boolean fields to reduce storage

Data Integrity: Maintained all original values while meeting specs

Comprehensive Validation: Checked value ranges and distributions



Cleaning for Economics Data

In [20]:
# 1. Create economics dataframe with exact column names from your DataFrame
economics = df[['client_id', 'cons_price_idx', 'euribor_three_months']].copy()


In [21]:
# 2. Data Validation Checks
print("Initial Data Validation:")
print("Data types:\n", economics.dtypes)
print("\nMissing values:\n", economics.isna().sum())
print("\nValue ranges:")
print("cons_price_idx:", economics['cons_price_idx'].min(), "-", economics['cons_price_idx'].max())
print("euribor_three_months:", economics['euribor_three_months'].min(), "-", economics['euribor_three_months'].max())



Initial Data Validation:
Data types:
 client_id                 int64
cons_price_idx          float64
euribor_three_months    float64
dtype: object

Missing values:
 client_id               0
cons_price_idx          0
euribor_three_months    0
dtype: int64

Value ranges:
cons_price_idx: 92.201 - 94.767
euribor_three_months: 0.634 - 5.045


In [22]:
# 3. Check for outliers
def detect_outliers(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return series[(series < lower_bound) | (series > upper_bound)]

print("\nPotential outliers in cons_price_idx:", len(detect_outliers(economics['cons_price_idx'])))
print("Potential outliers in euribor_three_months:", len(detect_outliers(economics['euribor_three_months'])))




Potential outliers in cons_price_idx: 0
Potential outliers in euribor_three_months: 0


In [23]:
# 4. Ensure proper data types
economics = economics.astype({
    'client_id': 'int64',
    'cons_price_idx': 'float64',
    'euribor_three_months': 'float64'
})



In [24]:
# 5. Final validation
print("\nFinal Data Validation:")
print("Data types:\n", economics.dtypes)
print("\nMissing values after cleaning:\n", economics.isna().sum())




Final Data Validation:
Data types:
 client_id                 int64
cons_price_idx          float64
euribor_three_months    float64
dtype: object

Missing values after cleaning:
 client_id               0
cons_price_idx          0
euribor_three_months    0
dtype: int64


In [25]:
# 6. Save to CSV
economics.to_csv('economics.csv', index=False)
print("\nCleaned economics data saved to economics.csv")


Cleaned economics data saved to economics.csv


Key Techniques:

Statistical Validation: Checked value distributions and ranges

Outlier Detection: Identified potential anomalies using IQR method

Type Consistency: Ensured proper numeric types for database

Minimal Transformation: Preserved original values as specified

Results Summary

Client Data

Processed 41,188 records

Standardized job/education categories

Converted 2 categorical fields to boolean

8,597 'unknown' credit defaults treated as 0

990 'unknown' mortgages treated as 0

Campaign Data

Created valid dates for all records

Converted 2 outcome fields to boolean

Contact duration range: 0-4,918 seconds

Campaign attempts range: 1-56 contacts

Economics Data

No missing values detected

No significant outliers found

CPI range: 92.201 - 94.767

EURIBOR range: 0.634 - 5.045

All three cleaned datasets were successfully saved as CSV files ready for database import.

Quality Assurance

Comprehensive validation at each transformation step

Maintained data integrity throughout cleaning

Documented all cleaning decisions

Preserved original values where specifications allowed

Optimized data types for database storage

